In [1]:
import pandas as pd
import ast
import re
import json
from sklearn import linear_model
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, r2_score, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from scipy import sparse

In [2]:
reviews = []
with open('scraped_reviews_fb_msngr_ytm.txt', 'r') as f: #, encoding='utf-8'
    reviews = f.read()

reviews = re.sub('\]', '', reviews)
reviews = re.sub('\[', '', reviews)
reviews = re.sub('\n', '', reviews)
reviews = re.sub("'", '"', reviews)
reviews = re.sub('    ', ' ', reviews)
reviews = re.sub(' {', '{', reviews)
reviews = re.sub(' }', '}', reviews)
reviews = re.sub('{ ', '{', reviews)
reviews = re.sub('} ', '}', reviews)
reviews = re.sub('}  {', '}, {', reviews)
reviews = re.sub('}{', '}, {', reviews)
reviews = re.sub('id:', '"id":', reviews)
reviews = re.sub('userName:', '"userName":', reviews)
reviews = re.sub('userImage:', '"userImage":', reviews)
reviews = re.sub('date:', '"date":', reviews)
reviews = re.sub('url:', '"url":', reviews)
reviews = re.sub('score:', '"score":', reviews)
reviews = re.sub('title:', '"title":', reviews)
reviews = re.sub('text:', '"text":', reviews)
reviews = re.sub('replyDate:', '"replyDate":', reviews)
reviews = re.sub('replyText:', '"replyText":', reviews)
reviews = re.sub(': undefined', ': "undefined"', reviews)

In [3]:
review_strings = [r+'}' for r in reviews.split('},')]
review_strings[-1] = review_strings[-1][:-1]

In [4]:
review_list = []
for r in review_strings:
    try:
        review_list.append(json.loads(r))
    except:
        pass

In [17]:
review_list[0][u'text']

u'It forces updates on you I will find another app'

In [6]:
review_text = [r['text'] for r in review_list]
review_dates = [r['date'] for r in review_list]
review_scores = [r['score'] for r in review_list]

In [7]:



sid = SentimentIntensityAnalyzer()

pscores = []
lens = []
for sentence in review_text:
    pscores.append(abs(sid.polarity_scores(sentence)['compound']))
    lens.append(np.log(len(sentence.split('.'))))
    
#pscores = np.reshape(pscores, [-1,1])

In [12]:
np.unique(review_dates)

array([u'October 1, 2018', u'October 10, 2018', u'October 11, 2018',
       u'October 12, 2018', u'October 13, 2018', u'October 14, 2018',
       u'October 15, 2018', u'October 2, 2018', u'October 5, 2018',
       u'October 6, 2018', u'October 7, 2018', u'October 8, 2018',
       u'October 9, 2018', u'September 22, 2018', u'September 23, 2018',
       u'September 25, 2018', u'September 26, 2018',
       u'September 27, 2018', u'September 28, 2018',
       u'September 29, 2018', u'September 30, 2018'], dtype='<U18')

<1810x2512 sparse matrix of type '<type 'numpy.int64'>'
	with 20457 stored elements in Compressed Sparse Row format>

In [37]:

import gensim
import nltk

import pandas as pd
from collections import Counter
import string

raw_corpus = []
for i in range(300): # change to larger number
    raw_corpus.append(review_list[i][u'text'])

texts = [[word.lower() for word in document.split(' ')]
         for document in raw_corpus]
from collections import Counter
freq = Counter()
for text in texts:
    freq.update(text)
processed_corpus = [[token for token in text if freq[token] > 1] for text in texts]
id2word = gensim.corpora.Dictionary(processed_corpus)
bow_corpus = [id2word.doc2bow(text) for text in processed_corpus]
tfidf = gensim.models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]




In [38]:
lda = gensim.models.ldamodel.LdaModel(corpus=tfidf_corpus, id2word=id2word,
                                      num_topics=20)

In [39]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis
vis_data = gensimvis.prepare( lda, tfidf_corpus, id2word)
pyLDAvis.display(vis_data)

In [33]:
# From here I'm not sure how to continue .... 

def normalize(k):
    k = k.lower()
    m = {"gans" : "gan",
     "gan" : "gan",
     "generative adversarial networks": "gan",
     "generative adversarial network": "gan",
      "recurrent neural networks": "rnn",
     "convolutional neural networks": "cnn",
     "nlu" : "nlp",
     "vae" : "variational inference",
     "natural language processing": "nlp",
     "lstm" : "rnn",
     "stochastic gradient descent": "sgd",
     "deep reinforcement learning": "reinforcement learning",
     "rl": "reinforcement learning",
     
    }
    return m.get(k, k)
notes = []
for i in range(101, 200): # change to larger number
    raw_corpus.append(review_list[i][u'text'])
keywords = Counter()
for n in notes:
    keywords.update(set([normalize(w) for w in n.content["keywords"]]))

marked = []    
for w, c in keywords.most_common(50):
    if w in ["deep learning", "neural network", "neural networks", 
             "machine learning"]: continue 
    marked.append(w)

In [34]:
def make_df(inp):
    df = []
    for i in range(len(inp)):
        topics = lda.get_document_topics(tfidf_corpus[i])
        topics.sort(key=lambda a: a[1])
        row = dict(title = inp[i].content["title"],
                       url = "https://openreview.net/forum?id=" + str(inp[i].id),
                       tldr = inp[i].content.get("TL;DR", ""),
                       topic = topics[0][0]
                      )
        for m in marked:
            row[m] = 1 if (m in [normalize(k) 
                                 for k in inp[i].content["keywords"]]) else 0
        df.append(row)
    return pd.DataFrame(df)
df = make_df(notes)

In [36]:
df

""
